# 1 Import Libraries

In [1]:
import pandas as pd

import numpy as np # library to handle data in a vectorized manner


import json # library to handle JSON files

!pip install geopy
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
!pip install geocoder

In [4]:
import geocoder # import geocoder

# **2 Download the South African cities data into a pandas dataframe**

In [5]:
data1 = pd.read_csv('SACities.csv')
data1.head()

,City,AccentCity,ProvinceName,Latitude,Longitude,ProvinceID
0,gqaka,Gqaka,Eastern Cape,-31.553917,28.210587,5
1,gqaqala,Gqaqala,Eastern Cape,-31.275246,28.422504,5
2,gqiba,Gqiba,Eastern Cape,-32.293342,28.686302,5
3,gqobonco,Gqobonco,Eastern Cape,-31.623649,28.255605,5
4,gqogqora,Gqogqora,Eastern Cape,-31.466667,28.533333,5


In [6]:
neighborhoods = data1.drop(['City', 'ProvinceID'], axis='columns')

neighborhoods.head()

,AccentCity,ProvinceName,Latitude,Longitude
0,Gqaka,Eastern Cape,-31.553917,28.210587
1,Gqaqala,Eastern Cape,-31.275246,28.422504
2,Gqiba,Eastern Cape,-32.293342,28.686302
3,Gqobonco,Eastern Cape,-31.623649,28.255605
4,Gqogqora,Eastern Cape,-31.466667,28.533333


In [7]:
neighborhood = neighborhoods.rename(columns = {'AccentCity': 'City'}, inplace = False)

neighborhood.head(5)

,City,ProvinceName,Latitude,Longitude
0,Gqaka,Eastern Cape,-31.553917,28.210587
1,Gqaqala,Eastern Cape,-31.275246,28.422504
2,Gqiba,Eastern Cape,-32.293342,28.686302
3,Gqobonco,Eastern Cape,-31.623649,28.255605
4,Gqogqora,Eastern Cape,-31.466667,28.533333


In [8]:
neighborhood.shape

(12675, 4)

# **3 Create a Gauteng Province only dataframe**

In [9]:
Gauteng = neighborhood.loc[neighborhood["ProvinceName"] == "Gauteng"]

In [10]:
Gauteng.head(10)

,City,ProvinceName,Latitude,Longitude
26,Graley,Gauteng,-25.766667,28.850000
33,Grasmere,Gauteng,-26.427497,27.860230
56,Greenview,Gauteng,-25.731389,28.397778
85,Groenpunt,Gauteng,-26.826341,28.038443
99,Groot-Elandsvlei,Gauteng,-26.139916,27.670300
207,Halfway House,Gauteng,-25.997137,28.129919
307,Hartsenbergfontein,Gauteng,-26.397421,27.956029
311,Hartzenbergfontein,Gauteng,-26.397421,27.956029
317,Hatfield,Gauteng,-25.749728,28.243141
318,Hatherley,Gauteng,-25.733333,28.366667


In [11]:
Gauteng.reset_index(drop=True, inplace=True)

In [12]:
Gauteng.head(10)

,City,ProvinceName,Latitude,Longitude
0,Graley,Gauteng,-25.766667,28.850000
1,Grasmere,Gauteng,-26.427497,27.860230
2,Greenview,Gauteng,-25.731389,28.397778
3,Groenpunt,Gauteng,-26.826341,28.038443
4,Groot-Elandsvlei,Gauteng,-26.139916,27.670300
5,Halfway House,Gauteng,-25.997137,28.129919
6,Hartsenbergfontein,Gauteng,-26.397421,27.956029
7,Hartzenbergfontein,Gauteng,-26.397421,27.956029
8,Hatfield,Gauteng,-25.749728,28.243141
9,Hatherley,Gauteng,-25.733333,28.366667


**Utilise Pretoria City to define a user agent as pt_explorer**

In [13]:
address = 'Pretoria, PT'

geolocator = Nominatim(user_agent="pt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pretoria are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pretoria are -25.7459374, 28.1879444.


# **4 Create a Gauteng Province map**

In [14]:
# create map of Gauteng using latitude and longitude values
map_Gauteng = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, ProvinceName, City in zip(Gauteng['Latitude'], Gauteng['Longitude'], Gauteng['ProvinceName'], Gauteng['City']):
    label = '{}, {}'.format(ProvinceName, City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Gauteng)  
    
map_Gauteng

# 5 Gauteng Venues Through Foursquare

In [15]:
CLIENT_ID = 'FKNRZCEZDMYK0BDUAWIFFS4PIT0J2NZDOK0EOE2NXG2Z5XYU' # your Foursquare ID
CLIENT_SECRET = 'UMV202PC25CFGU3A5C45WGBGMSELYESJMA2W4NMUE12I5LYM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FKNRZCEZDMYK0BDUAWIFFS4PIT0J2NZDOK0EOE2NXG2Z5XYU
CLIENT_SECRET:UMV202PC25CFGU3A5C45WGBGMSELYESJMA2W4NMUE12I5LYM


**Explore the first neighborhood in the Gauteng Province**

In [16]:
Gauteng.loc[8, 'City']

'Hatfield'

In [17]:
neighborhood_latitude = Gauteng.loc[8, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Gauteng.loc[8, 'Longitude'] # neighborhood longitude value

neighborhood_name = Gauteng.loc[8, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hatfield are -25.749728, 28.243140999999998.


# 6 Lets get the top 100 venues in Hatfield within a 500 meter radius

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FKNRZCEZDMYK0BDUAWIFFS4PIT0J2NZDOK0EOE2NXG2Z5XYU&client_secret=UMV202PC25CFGU3A5C45WGBGMSELYESJMA2W4NMUE12I5LYM&v=20180605&ll=-25.749728,28.243140999999998&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6006f65214532e7aafe5c378'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Pretoria',
  'headerFullLocation': 'Pretoria',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': -25.745227995499995,
    'lng': 28.24812778913279},
   'sw': {'lat': -25.754228004500007, 'lng': 28.238154210867204}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd875dbf645c9b6a1fca7e0',
       'name': 'Bravo Pizzeria',
       'location': {'address': '1212 South Street',
        'crossStreet': 'Jan Shoba (Duncan)',
        'lat': -25.753017471122593,
        'lng': 28.24088926346206,
        'labeledLatLngs': [{'label': 'di

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-21-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Bravo Pizzeria,Pizza Place,-25.753017,28.240889
1,"Moloko, Pretoria",Nightclub,-25.752554,28.240059
2,Deli On Duncan,Deli / Bodega,-25.751315,28.240496
3,Papa's Real Food & Function House,Restaurant,-25.751364,28.240683
4,+27 Café,Café,-25.752788,28.239961


Lets check how many venues were returned by Foursquare

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


# 7 Now lets Explore Gauteng Neighborhoods

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
gauteng_venues = getNearbyVenues(names=Gauteng['City'],
                                   latitudes=Gauteng['Latitude'],
                                   longitudes=Gauteng['Longitude']
                                  )

Graley
Grasmere
Greenview
Groenpunt
Groot-Elandsvlei
Halfway House
Hartsenbergfontein
Hartzenbergfontein
Hatfield
Hatherley
Hatherly
Heather Bell
Heidelberg
Heidelburg
Hekpoort
Zesfontein
Zonderwater
Zonnehoeve
Zusterstroom
Springs
Spykerras
Stil Gelee
Stofberg-Gedenkskool
Stofberg
Stoffberggedenkskool
Strathavon
Strydfontein
Sunnyside
Susterstroom
Swartspruit
Sybrandskraal
Tarlton
Tembisa
Ten Acre
Tetema
Thabakhubedu
The Highlands
The Village
Thorndale
Three Rivers
Tierpoort
Toekomsrus
Tokoza
Trevallyn
Triomf
Tsakane
Turffontein
Vaalplaas
Vaalview
Valhalla
Valley Nedersettings
Valley Settlements
Vanderbijlpark
Van der Merweskroon
Van der Merwe
Van Wyksrust
Van Wyksrus
Vasfontein
Venterpos-West
Venterspost
Venterspos
Vereeniging
Verena
Verwoerdburg
Villieria
Vlakplaas
Vlakplaats
Vleikop
Voortrekkerhoogte
Voortrekkers
Voortrekker
Vosloorus
Vrededorp
Waaikraal
Walkerville
Wallmannstal
Wallmannsthal
Wallmansthall
Waltloo
Waterdal
Waterkloof Glen
Waterkloof
Water's Edge
Khutsong
Kings Kloo

**Let's find out how many unique categories can be curated from all the returned venues**

In [27]:
print('There are {} uniques categories.'.format(len(gauteng_venues['Venue Category'].unique())))

There are 161 uniques categories.


Lets check the venues dataframe

In [28]:
print(gauteng_venues.shape)
gauteng_venues.head()

(839, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Halfway House,-25.997137,28.129919,BP,-25.995249,28.128440,Gas Station
1,Halfway House,-25.997137,28.129919,Chicken Licken,-25.996536,28.126773,Fried Chicken Joint
2,Halfway House,-25.997137,28.129919,Nando's,-25.996764,28.127478,Portuguese Restaurant
3,Halfway House,-25.997137,28.129919,KFC,-25.997886,28.126834,Fast Food Restaurant
4,Halfway House,-25.997137,28.129919,Virgin Active,-25.996378,28.126123,Gym / Fitness Center


In [ ]:
gauteng_venues.groupby('Neighborhood').count()

# 8 Lets Analyze each neighborhood

In [29]:
# one hot encoding
gauteng_onehot = pd.get_dummies(gauteng_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
gauteng_onehot['Neighborhood'] = gauteng_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [gauteng_onehot.columns[-1]] + list(gauteng_onehot.columns[:-1])
gauteng_onehot = gauteng_onehot[fixed_columns]

gauteng_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Tree,Turkish Restaurant,Vape Store,Video Store,Water Park,Wine Shop,Wings Joint,Yoga Studio
0,Halfway House,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Halfway House,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Halfway House,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Halfway House,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Halfway House,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
gauteng_onehot.shape

(839, 162)

**Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [31]:
gauteng_grouped = gauteng_onehot.groupby('Neighborhood').mean().reset_index()
gauteng_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Antique Shop,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Tree,Turkish Restaurant,Vape Store,Video Store,Water Park,Wine Shop,Wings Joint,Yoga Studio
0,Akasia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alberton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Albertynsville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alexandra,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Annlin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wingate Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250,Witfield,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,Wynberg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,Yeoville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in gauteng_onehot['Neighborhood']:
    print("----"+hood+"----")
    temp = gauteng_onehot[gauteng_onehot['Neighborhood'] == hood].T.reset_index()
    #temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    #temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues = pd.DataFrame(columns=columns)
neighborhoods_venues['Neighborhood'] = gauteng_grouped['Neighborhood']

for ind in np.arange(gauteng_grouped.shape[0]):
    neighborhoods_venues.iloc[ind, 1:] = return_most_common_venues(gauteng_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akasia,Construction & Landscaping,Yoga Studio,Farm,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,Alberton,German Restaurant,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,Albertynsville,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
3,Alexandra,Afghan Restaurant,Butcher,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Annlin,Hotel,Hostel,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


# 9 Lets Cluster Gauteng Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [32]:
# set number of clusters
kclusters = 5

gauteng_clustering = gauteng_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(gauteng_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 3, 0, 0, 0, 0, 1, 0, 0])

Create a dataframe that includes the cluster as well as the top 10 venues for each Gauteng neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)

gauteng_merged = Gauteng

# merge gauteng_grouped with Gauteng data to add latitude/longitude for each neighborhood
gauteng_merged = gauteng_merged.join(neighborhoods_venues)

gauteng_merged.head()

,City,ProvinceName,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Graley,Gauteng,-25.766667,28.850000,2.0,Akasia,Construction & Landscaping,Yoga Studio,Farm,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,Grasmere,Gauteng,-26.427497,27.860230,0.0,Alberton,German Restaurant,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,Greenview,Gauteng,-25.731389,28.397778,3.0,Albertynsville,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
3,Groenpunt,Gauteng,-26.826341,28.038443,0.0,Alexandra,Afghan Restaurant,Butcher,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Groot-Elandsvlei,Gauteng,-26.139916,27.670300,0.0,Annlin,Hotel,Hostel,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

gauteng_merged['Cluster Labels'] = gauteng_merged['Cluster Labels'].fillna(0).astype(np.int64)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gauteng_merged['Latitude'], gauteng_merged['Longitude'], gauteng_merged['Neighborhood'], gauteng_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 10 Lets Examine Each Cluster

Lets examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, each cluster is assigned a number.

In [39]:
gauteng_merged.loc[gauteng_merged['Cluster Labels'] == 0, gauteng_merged.columns[[1] + list(range(5, gauteng_merged.shape[1]))]]

,ProvinceName,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Gauteng,Alberton,German Restaurant,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
3,Gauteng,Alexandra,Afghan Restaurant,Butcher,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Gauteng,Annlin,Hotel,Hostel,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
5,Gauteng,Arcon Park,Hotel,Hostel,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
6,Gauteng,Aston Woods,Lake,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
...,...,...,...,...,...,...,...,...,...,...,...,...
485,Gauteng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,Gauteng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,Gauteng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488,Gauteng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 2

In [48]:
gauteng_merged.loc[gauteng_merged['Cluster Labels'] == 1, gauteng_merged.columns[[1] + list(range(5, gauteng_merged.shape[1]))]]

,ProvinceName,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Gauteng,Atteridgeville,Pizza Place,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
36,Gauteng,Culemborgpark,Pizza Place,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
166,Gauteng,Oberholzer,Pizza Place,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
191,Gauteng,Randfontein,Pizza Place,Dance Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 3

In [45]:
gauteng_merged.loc[gauteng_merged['Cluster Labels'] == 2, gauteng_merged.columns[[1] + list(range(5, gauteng_merged.shape[1]))]]

,ProvinceName,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Gauteng,Akasia,Construction & Landscaping,Yoga Studio,Farm,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
16,Gauteng,Boipatong,Home Service,Yoga Studio,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
42,Gauteng,De Deur Estates,Construction & Landscaping,Yoga Studio,Farm,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
65,Gauteng,Erasmia,Convenience Store,Construction & Landscaping,Yoga Studio,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
72,Gauteng,Fietas,Home Service,Construction & Landscaping,Business Service,Yoga Studio,Farm,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
88,Gauteng,Hekpoort,Campground,Construction & Landscaping,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
106,Gauteng,Kew,Home Service,Yoga Studio,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
131,Gauteng,Mabopane,Home Service,Yoga Studio,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
137,Gauteng,Marshallstown,Home Service,Construction & Landscaping,Business Service,Yoga Studio,Farm,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
138,Gauteng,Marshalltown,Home Service,Construction & Landscaping,Business Service,Yoga Studio,Farm,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


Cluster 4

In [42]:
gauteng_merged.loc[gauteng_merged['Cluster Labels'] == 3, gauteng_merged.columns[[1] + list(range(5, gauteng_merged.shape[1]))]]

,ProvinceName,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gauteng,Albertynsville,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
52,Gauteng,Duncanville,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
67,Gauteng,Evaton,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
82,Gauteng,Glenharvie,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
102,Gauteng,Katlehong,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
144,Gauteng,Mid-Ennerdale,Convenience Store,Gas Station,Yoga Studio,Farm,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
145,Gauteng,Middenerdale,Convenience Store,Gas Station,Yoga Studio,Farm,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
149,Gauteng,Mohlakeng,Pub,Gas Station,Yoga Studio,Factory,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
194,Gauteng,Rietfontein,Gas Station,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 5

In [46]:
gauteng_merged.loc[gauteng_merged['Cluster Labels'] == 4, gauteng_merged.columns[[1] + list(range(5, gauteng_merged.shape[1]))]]

,ProvinceName,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Gauteng,Booysens,Fried Chicken Joint,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
104,Gauteng,Kenilworth,Fried Chicken Joint,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
120,Gauteng,Lenasia,Fried Chicken Joint,Bakery,Yoga Studio,Farmers Market,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
185,Gauteng,Primrose,Fried Chicken Joint,Burger Joint,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
